## Data Ingestion

In [45]:
from datetime import date
import json
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

# pandas options
pd.options.display.float_format = '{:,.2f}'.format

# USE CAUTION/UNCOMMENT if necessary
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 30)
# pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 150)

# plotting
%matplotlib inline

# Initial dataframe is the top-level, need to flatten
df_initial = pd.read_json('top_wallets_and_transactions_with_USD_prices.json')
display(df_initial.head())
print(df_initial.shape)
print(df_initial.columns)

,holders
0,"{'address': '0x2f2456953a2cbc21fde058d33fb2ad8d59e72a35', 'balance': 1108241744.15, 'share': 36.94, 'transactions': [{'timestamp': 1630991473, 'tr..."
1,"{'address': '0x5f5b8942ae6325227d5d82e5759e837271fa2a67', 'balance': 522999999.99999994, 'share': 17.43, 'transactions': [{'timestamp': 1628783320..."
2,"{'address': '0x8ffa64fb50559c3ff09a1022b84b2c5233ed8068', 'balance': 281512136.62667745, 'share': 9.38, 'transactions': [{'timestamp': 1625488642,..."
3,"{'address': '0xf977814e90da44bfa03b6295a0616a897441acec', 'balance': 261899174.6018669, 'share': 8.73, 'transactions': [{'timestamp': 1636830622, ..."
4,"{'address': '0x07d2601739709c25fe0afd50ec961ba589311caa', 'balance': 195000000.0, 'share': 6.5, 'transactions': [{'timestamp': 1628789705, 'transa..."


(200, 1)
Index(['holders'], dtype='object')


In [46]:
def flat_sand_data():
    '''Flatten JSON object and normalize to transaction level data.'''
    with open('top_wallets_and_transactions_with_USD_prices.json','r') as json_file:
        json_data = json.load(json_file)
        df = pd.json_normalize(
            json_data,
            record_path=["holders","transactions"],
            sep="_",
            meta=[
                 ['holders','address'],
                 ['holders','balance'],
                 ['holders','share'],
            ],
            errors='ignore'
        )
        df = df.rename(
            columns={
                'timestamp': 'date_time',
                'type': 'tx_type',
                'from': 'address_from',
                'to': 'address_to',
                'value': 'abs_value'
            }
        )
        df['date_time'] = pd.to_datetime(df['date_time'], unit='s') # Convert from UNIX timestamp.
        df['value'] = np.where(df['holders_address']!=df['address_from'], -df['abs_value'], df['abs_value']) # !!!Positive values are sales, Negative values indicate purchase.
        df['value_in_USD'] = df['value'] * df['USD_price_at_timestamp'] # Cash Flow data in USD.
        df['uniq_transID'] = df['holders_address'] + df['transactionHash']
        
        df = df.reindex(
            columns = [
                'holders_address',
                'holders_balance', 
                'holders_share',
                'uniq_transID',
                'transactionHash',
                'date_time',
                'tx_type',
                'address_from',
                'address_to',
                'abs_value',
                'value',
                'USD_price_at_timestamp',
                'value_in_USD'
            ]
        )
        
    return df

df = flat_sand_data()
df.to_csv('df_dataframe.csv', header=True) # Useful to view DF as CSV.
df.head()




,holders_address,holders_balance,holders_share,uniq_transID,transactionHash,date_time,tx_type,address_from,address_to,abs_value,value,USD_price_at_timestamp,value_in_USD
0,0x2f2456953a2cbc21fde058d33fb2ad8d59e72a35,"1,108,241,744.15",36.94,0x2f2456953a2cbc21fde058d33fb2ad8d59e72a350x50487f46f22f02c0bfae873f28065bd30ec9f904911f905287a1cd04a2dedbdd,0x50487f46f22f02c0bfae873f28065bd30ec9f904911f905287a1cd04a2dedbdd,2021-09-07 05:11:13,transfer,0x2f2456953a2cbc21fde058d33fb2ad8d59e72a35,0x2396c520895ddb998c48acce50a5f6de33ecc5f9,"6,000,000.00","6,000,000.00",0.87,"5,234,235.38"
1,0x2f2456953a2cbc21fde058d33fb2ad8d59e72a35,"1,108,241,744.15",36.94,0x2f2456953a2cbc21fde058d33fb2ad8d59e72a350xe1f1657df50ebb6b74f38d06570bd28ae5861efb3b19451f083fabad44a5bd3f,0xe1f1657df50ebb6b74f38d06570bd28ae5861efb3b19451f083fabad44a5bd3f,2021-09-02 22:22:10,transfer,0x2f2456953a2cbc21fde058d33fb2ad8d59e72a35,0x2f8c8e82185c831ea96c02edda9dfac941ac98e3,"806,182.00","806,182.00",1.01,"811,613.97"
2,0x2f2456953a2cbc21fde058d33fb2ad8d59e72a35,"1,108,241,744.15",36.94,0x2f2456953a2cbc21fde058d33fb2ad8d59e72a350x5f5c489f9255598d4422114b2a7002de8d71995c61621e47c6a15b5569fbe68a,0x5f5c489f9255598d4422114b2a7002de8d71995c61621e47c6a15b5569fbe68a,2021-08-18 17:30:00,transfer,0x2f2456953a2cbc21fde058d33fb2ad8d59e72a35,0xac8eb5574c9c706f5293a247bdb5d667e87a0599,"1,388,889.00","1,388,889.00",0.59,"825,238.84"
3,0x2f2456953a2cbc21fde058d33fb2ad8d59e72a35,"1,108,241,744.15",36.94,0x2f2456953a2cbc21fde058d33fb2ad8d59e72a350x2dd8630d7cf54ea63a9adccca0d8a44d4c35fa0c01185733c59ab6f0dbdcfb29,0x2dd8630d7cf54ea63a9adccca0d8a44d4c35fa0c01185733c59ab6f0dbdcfb29,2021-08-18 05:35:14,transfer,0x2f2456953a2cbc21fde058d33fb2ad8d59e72a35,0xdf89c888c802eafaad084e9b1d5298409888ee3f,"1,122,094.00","1,122,094.00",0.59,"666,716.74"
4,0x2f2456953a2cbc21fde058d33fb2ad8d59e72a35,"1,108,241,744.15",36.94,0x2f2456953a2cbc21fde058d33fb2ad8d59e72a350x80421643f22411b2322e5927636a6e600f90decc2acbda7714129caaa7095ff9,0x80421643f22411b2322e5927636a6e600f90decc2acbda7714129caaa7095ff9,2021-08-17 13:11:43,transfer,0x2f2456953a2cbc21fde058d33fb2ad8d59e72a35,0xc34003b336899ece931075c580a2b45b8c3a4f72,"336,628.00","336,628.00",0.64,"216,225.09"


In [47]:
df.columns

Index(['holders_address', 'holders_balance', 'holders_share', 'uniq_transID',
       'transactionHash', 'date_time', 'tx_type', 'address_from', 'address_to',
       'abs_value', 'value', 'USD_price_at_timestamp', 'value_in_USD'],
      dtype='object')

In [57]:
df[df['holders_address']=='0x326d9f47ba49bbaac279172634827483af70a601']

,holders_address,holders_balance,holders_share,uniq_transID,transactionHash,date_time,tx_type,address_from,address_to,abs_value,value,USD_price_at_timestamp,value_in_USD
4285,0x326d9f47ba49bbaac279172634827483af70a601,"10,600,005.00",0.35,0x326d9f47ba49bbaac279172634827483af70a6010x904cebc65859285ad0de81c0f3dc66b020fb5ed8e5df395e7ce5e64e6810d659,0x904cebc65859285ad0de81c0f3dc66b020fb5ed8e5df395e7ce5e64e6810d659,2021-11-24 16:53:26,transfer,0xb15bfc3c04a726dc6df1bd8d658e1a1e43a9af33,0x326d9f47ba49bbaac279172634827483af70a601,"5,000,000.00","-5,000,000.00",7.59,"-37,952,875.44"
4286,0x326d9f47ba49bbaac279172634827483af70a601,"10,600,005.00",0.35,0x326d9f47ba49bbaac279172634827483af70a6010x48af77bea86433814e7fc511b6138249fb5b727e28e5113f4e6df7f01c086c7a,0x48af77bea86433814e7fc511b6138249fb5b727e28e5113f4e6df7f01c086c7a,2021-11-23 16:37:33,transfer,0x79169e7818968cd0c6dbd8929f24d797cc1af9a1,0x326d9f47ba49bbaac279172634827483af70a601,"1,500,000.00","-1,500,000.00",5.20,"-7,800,268.68"
4287,0x326d9f47ba49bbaac279172634827483af70a601,"10,600,005.00",0.35,0x326d9f47ba49bbaac279172634827483af70a6010x1941911ad0c704543a3490b2a2cbdcc69d047b48cf8e3d934dede014ec130c24,0x1941911ad0c704543a3490b2a2cbdcc69d047b48cf8e3d934dede014ec130c24,2021-11-22 21:50:56,transfer,0x326d9f47ba49bbaac279172634827483af70a601,0x79169e7818968cd0c6dbd8929f24d797cc1af9a1,"3,000,000.00","3,000,000.00",4.61,"13,815,980.14"
4288,0x326d9f47ba49bbaac279172634827483af70a601,"10,600,005.00",0.35,0x326d9f47ba49bbaac279172634827483af70a6010x8bb84537c81ff92542ca449b2e79d78c04c001ed51810bd92b02233d7d75a706,0x8bb84537c81ff92542ca449b2e79d78c04c001ed51810bd92b02233d7d75a706,2021-11-18 16:35:15,transfer,0xb15bfc3c04a726dc6df1bd8d658e1a1e43a9af33,0x326d9f47ba49bbaac279172634827483af70a601,"6,800,000.00","-6,800,000.00",4.06,"-27,576,488.27"
4289,0x326d9f47ba49bbaac279172634827483af70a601,"10,600,005.00",0.35,0x326d9f47ba49bbaac279172634827483af70a6010xaa808c9aef96a68503593cd8e0c0af632f76749446049d329fa59839374e7629,0xaa808c9aef96a68503593cd8e0c0af632f76749446049d329fa59839374e7629,2021-11-17 15:58:41,transfer,0x326d9f47ba49bbaac279172634827483af70a601,0x79169e7818968cd0c6dbd8929f24d797cc1af9a1,"2,000,000.00","2,000,000.00",3.26,"6,517,889.44"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4300,0x326d9f47ba49bbaac279172634827483af70a601,"10,600,005.00",0.35,0x326d9f47ba49bbaac279172634827483af70a6010x2a8994991947db0b1e8b924af2fdf0d6826bd400cc48bc231956b0d5cab1d184,0x2a8994991947db0b1e8b924af2fdf0d6826bd400cc48bc231956b0d5cab1d184,2021-11-04 16:42:06,transfer,0x79169e7818968cd0c6dbd8929f24d797cc1af9a1,0x326d9f47ba49bbaac279172634827483af70a601,"500,000.00","-500,000.00",2.72,"-1,359,914.49"
4301,0x326d9f47ba49bbaac279172634827483af70a601,"10,600,005.00",0.35,0x326d9f47ba49bbaac279172634827483af70a6010xb97558616c8980de3092be839e5f1b32a3a8c1760ec0a7fa682f8958a7e5e48b,0xb97558616c8980de3092be839e5f1b32a3a8c1760ec0a7fa682f8958a7e5e48b,2021-11-04 15:49:47,transfer,0x79169e7818968cd0c6dbd8929f24d797cc1af9a1,0x326d9f47ba49bbaac279172634827483af70a601,"1,000,000.00","-1,000,000.00",2.76,"-2,764,395.37"
4302,0x326d9f47ba49bbaac279172634827483af70a601,"10,600,005.00",0.35,0x326d9f47ba49bbaac279172634827483af70a6010x922d22a4d38a2663d383856d1c583e8c5851d4aa076f24624ade2cee1b4911f4,0x922d22a4d38a2663d383856d1c583e8c5851d4aa076f24624ade2cee1b4911f4,2021-11-02 14:16:21,transfer,0x326d9f47ba49bbaac279172634827483af70a601,0x79169e7818968cd0c6dbd8929f24d797cc1af9a1,"5,000,000.00","5,000,000.00",2.65,"13,255,850.96"
4303,0x326d9f47ba49bbaac279172634827483af70a601,"10,600,005.00",0.35,0x326d9f47ba49bbaac279172634827483af70a6010xeb27d55aa4884a2c2f3c65069f755a9bab911ddd442a5bd5d9b90700f8ae34ae,0xeb27d55aa4884a2c2f3c65069f755a9bab911ddd442a5bd5d9b90700f8ae34ae,2021-11-01 16:37:25,transfer,0xb15bfc3c04a726dc6df1bd8d658e1a1e43a9af33,0x326d9f47ba49bbaac279172634827483af70a601,"8,000,000.00","-8,000,000.00",1.83,"-14,656,695.15"


## Work In Progress (Wallet-Level Data):

In [59]:
wallet_level_data = df[['holders_address','holders_balance','holders_share','transactionHash','uniq_transID','date_time']].groupby('holders_address').min()
wallet_level_data = wallet_level_data.rename(
            columns={
                'transactionHash': 'initial_tx_hash',
                'date_time': 'initial_date'
            }
        )
wallet_level_data['holding_period_month'] = (pd.Timestamp(date.today()) - wallet_level_data['initial_date']).astype('timedelta64[M]')

wallet_level_data = pd.merge(wallet_level_data, df, on="uniq_transID",how='inner')
wallet_level_data.rename(
            columns={
                'holders_balance_x': 'holders_balance',
                'holders_share_x': 'holders_share',
                'value': 'abs_value'
            }
        )
wallet_level_data.reindex(
            columns = [
                'holders_balance',
                'holders_share', 
                'holders_address',
                'initial_tx_hash',
                'initial_date',
                'holding_period_month',
                'tx_type',
                'address_from',
                'address_to',
                'abs_value',
                'value',
                'USD_price_at_timestamp',
                'value_in_USD'
                'holding_period_month'
            ]
        )
wallet_level_data.drop(['holders_balance_y', 'holders_share_y','transactionHash'], axis=1, inplace=True)
display(wallet_level_data)

wallet_level_data.to_csv('wallet_level_data.csv', header=True) # Useful to view DF as CSV.

,holders_balance_x,holders_share_x,initial_tx_hash,uniq_transID,initial_date,holding_period_month,holders_address,date_time,tx_type,address_from,address_to,abs_value,value,USD_price_at_timestamp,value_in_USD
0,"279,251.22",0.01,0x84a3d0a06046d96dd2969d288d0f0a752e992fe040a83312c48f2dfe855024a3,0x00b9e69051e0b5787b8d2a9a0308e415294e6b8a0x84a3d0a06046d96dd2969d288d0f0a752e992fe040a83312c48f2dfe855024a3,2021-10-16 18:19:35,1.00,0x00b9e69051e0b5787b8d2a9a0308e415294e6b8a,2021-10-16 18:19:35,transfer,0x9d68c9dfbeca4a0d2a35365f01b90e8626af4417,0x00b9e69051e0b5787b8d2a9a0308e415294e6b8a,"279,251.22","-279,251.22",0.74,"-207,564.59"
1,"604,495.41",0.02,0x013899c638efe612b6e0ce09ce71014fe1c92448368899848f51c790bbc678a2,0x0211f3cedbef3143223d3acf0e589747933e85270x013899c638efe612b6e0ce09ce71014fe1c92448368899848f51c790bbc678a2,2021-11-19 14:12:03,0.00,0x0211f3cedbef3143223d3acf0e589747933e8527,2021-11-19 14:12:03,transfer,0x579d30bd4ea860c40698b4c2a85bf99e21ad6e1a,0x0211f3cedbef3143223d3acf0e589747933e8527,"2,908.28","-2,908.28",4.35,"-12,639.11"
2,"110,480,542.03",3.68,0x0010d886029bd8082cd3b5afb65ee6af9bc96f3622c8cc09418f8814d84eccaa,0x0404d96d2583799bc73edb8d230aa037fc22b3790x0010d886029bd8082cd3b5afb65ee6af9bc96f3622c8cc09418f8814d84eccaa,2021-07-15 09:11:20,4.00,0x0404d96d2583799bc73edb8d230aa037fc22b379,2021-07-22 02:10:42,transfer,0x1938a448d105d26c40a52a1bfe99b8ca7a745ad0,0x0404d96d2583799bc73edb8d230aa037fc22b379,"87,796.94","-87,796.94",0.49,"-42,752.26"
3,"2,205,375.00",0.07,0x210750b99b9ee23a36730b24e8fa5f07572f16fd6ea9e2ed45b304c0b983ea2c,0x07445065963c2d563cd70ddf2aa49fc771e59a980x210750b99b9ee23a36730b24e8fa5f07572f16fd6ea9e2ed45b304c0b983ea2c,2021-11-22 06:23:10,0.00,0x07445065963c2d563cd70ddf2aa49fc771e59a98,2021-11-22 06:23:10,transfer,0xd9d8d445f4e578ffb68349ff49186d6edc97b4f1,0x07445065963c2d563cd70ddf2aa49fc771e59a98,"1,770,000.00","-1,770,000.00",3.92,"-6,943,437.70"
4,"195,000,000.00",6.50,0x13c09d9c43a16c92438f309a32601cd16be88aa6ec0d97b8c3c48a195db24800,0x07d2601739709c25fe0afd50ec961ba589311caa0x13c09d9c43a16c92438f309a32601cd16be88aa6ec0d97b8c3c48a195db24800,2020-08-18 17:42:23,15.00,0x07d2601739709c25fe0afd50ec961ba589311caa,2021-08-12 16:20:20,transfer,0x07d2601739709c25fe0afd50ec961ba589311caa,0x5366dc49e76ba0173921c636aa23dcd00e4b296b,"9,000,000.00","9,000,000.00",0.65,"5,880,235.74"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198,"1,646,379.81",0.05,0x0ef2368ff3ce025f804be844e7d7b8a5341f4e1da288b4a18a42730d2cd0fe2f,0xfbb1b73c4f0bda4f67dca266ce6ef42f520fbb980x0ef2368ff3ce025f804be844e7d7b8a5341f4e1da288b4a18a42730d2cd0fe2f,2021-11-24 10:08:11,0.00,0xfbb1b73c4f0bda4f67dca266ce6ef42f520fbb98,2021-11-24 19:13:48,transfer,0xfbb1b73c4f0bda4f67dca266ce6ef42f520fbb98,0x287e6d4dd4d57ede08b9353e13d51d9f7f1f95fa,"11,629.01","11,629.01",7.64,"88,790.80"
199,"170,000.00",0.01,0x17bc246c7cbe69f2484878d79d7e6e18b92d49984d0c589d3d4ff4c73335b376,0xfd80c78b3f755a25a7698aa335b2b6a8ca230e840x17bc246c7cbe69f2484878d79d7e6e18b92d49984d0c589d3d4ff4c73335b376,2021-10-06 03:34:00,1.00,0xfd80c78b3f755a25a7698aa335b2b6a8ca230e84,2021-10-06 06:37:23,transfer,0xc442969e7e56df73c173d4b97e56a37054f48aae,0xfd80c78b3f755a25a7698aa335b2b6a8ca230e84,"30,000.00","-30,000.00",0.80,"-24,057.21"
200,"1,048,005.46",0.03,0x184faaf3cc6a49d98d94cc50452ecf9fcbf4ce95bd888cef55c935cc64cb275d,0xfed694dc2ce345dc23a5a915cdb5d55b258351c90x184faaf3cc6a49d98d94cc50452ecf9fcbf4ce95bd888cef55c935cc64cb275d,2021-10-05 06:26:03,1.00,0xfed694dc2ce345dc23a5a915cdb5d55b258351c9,2021-10-05 06:31:26,transfer,0x28c6c06298d514db089934071355e5743bf21d60,0xfed694dc2ce345dc23a5a915cdb5d55b258351c9,"736,635.48","-736,635.48",0.81,"-599,849.87"
201,"854,380.67",0.03,0x051c985af67b437e0a744197848b3ab61510263f85ff3b5d262cdd7322128aa8,0xff0cb0351a356ad16987e5809a8daaaf34f5adbe0x051c985af67b437e0a744197848b3ab61510263f85ff3b5d262cdd7322128aa8,2021-07-05 04:16:58,4.00,0xff0cb0351a356ad16987e5809a8daaaf34f5adbe,2021-11-01 12:09:36,transfer,0x74de5d4fcbf63e00

# Sanity Checks

In [106]:
print(df.columns)

# Only pulls in transaction type: transfer.
print(f'\n\nThe type of transaction pulled:')
display(df.value_counts(['tx_type']))

# Date Range, seems reasonable
print(f'\n\nThe first date of date range is:')
display(df['date_time'].min())
print(f'\n\nThe last date of date range is:')
display(df['date_time'].max())

# Any same to/from addresses?
print(f'\n\nWere there transactions where the tokens were transfer from one wallet to itself?')
display(df.value_counts(df['address_from']==df['address_to']))
                       
# 18k transactions in line with Ethan's estimates.
print(f'\n\nThe total number of transactions in the dataset:')
print(f'Shape: {df.shape}')


Index(['holders_address', 'holders_balance', 'holders_share', 'uniq_transID',
       'transactionHash', 'date_time', 'tx_type', 'address_from', 'address_to',
       'abs_value', 'value', 'USD_price_at_timestamp', 'value_in_USD'],
      dtype='object')


The type of transaction pulled:


tx_type 
transfer    18139
dtype: int64



The first date of date range is:


Timestamp('2020-08-05 10:27:29')



The last date of date range is:


Timestamp('2021-11-24 20:38:01')



Were there transactions where the tokens were transfer from one wallet to itself?


False    18139
dtype: int64



The total number of transactions in the dataset:
Shape: (18139, 13)


In [107]:
# The sum of the holders_share should be less than 100%
print(f'\n\nThe sum of the holders_share is:')
total_share = df[['holders_address', 'holders_share']].groupby('holders_address').max().sort_values('holders_share', ascending=False)
display(total_share['holders_share'].sum())


# The sum of the balance should be less than 100% of the total # of circulating tokens
print(f'\n\nThe sum of the holders_balance is:')
total_balance = df[['holders_address', 'holders_balance']].groupby('holders_address').max().sort_values('holders_balance', ascending=False)
display(total_balance['holders_balance'].sum())


# The USD price at each date_time stamp
print(f'\n\nThe USD price at each date_time stamp:')
display(df[['date_time', 'USD_price_at_timestamp']].sort_values('date_time'))


# Count NaN or empty values
print(f'\n\nThe count of NaN or empty values:')
display(df.isnull().sum())


# Why were these value_in_USD NaN or empty? Some of the timestamps are in 2021
print(f'\n\nThe rows containing NaN or empty values:')
display(df.loc[df['value_in_USD'].isnull()])

# df_null.to_csv('export_dataframe.csv', index = False, header=True) # Useful to open DF as CSV.




The sum of the holders_share is:


97.41000000000001



The sum of the holders_balance is:


2916123141.7947025



The USD price at each date_time stamp:


,date_time,USD_price_at_timestamp
28,2020-08-05 10:27:29,NaN
27,2020-08-11 23:08:29,NaN
4284,2020-08-11 23:08:29,NaN
26,2020-08-12 11:01:48,NaN
6768,2020-08-13 20:43:14,NaN
...,...,...
4610,2021-11-24 20:36:55,7.37
5365,2021-11-24 20:36:55,7.37
4608,2021-11-24 20:37:14,7.46
15356,2021-11-24 20:37:40,7.46




The count of NaN or empty values:


holders_address            0
holders_balance            0
holders_share              0
uniq_transID               0
transactionHash            0
date_time                  0
tx_type                    0
address_from               0
address_to                 0
abs_value                  0
value                      0
USD_price_at_timestamp    11
value_in_USD              11
dtype: int64



The rows containing NaN or empty values:


,holders_address,holders_balance,holders_share,uniq_transID,transactionHash,date_time,tx_type,address_from,address_to,abs_value,value,USD_price_at_timestamp,value_in_USD
25,0x2f2456953a2cbc21fde058d33fb2ad8d59e72a35,"1,108,241,744.15",36.94,0x2f2456953a2cbc21fde058d33fb2ad8d59e72a350x186374b502fbe1e1f7842c6f603579ce221ae674f8be9e907c39d3c7a85ca4d4,0x186374b502fbe1e1f7842c6f603579ce221ae674f8be9e907c39d3c7a85ca4d4,2020-08-13 20:43:14,transfer,0x2f2456953a2cbc21fde058d33fb2ad8d59e72a35,0x5366dc49e76ba0173921c636aa23dcd00e4b296b,"50,000,000.00","50,000,000.00",NaN,NaN
26,0x2f2456953a2cbc21fde058d33fb2ad8d59e72a35,"1,108,241,744.15",36.94,0x2f2456953a2cbc21fde058d33fb2ad8d59e72a350x29f57014c89f919ef944f930b130b29e78811af14c8ad8254eae866548b97dd8,0x29f57014c89f919ef944f930b130b29e78811af14c8ad8254eae866548b97dd8,2020-08-12 11:01:48,transfer,0x2f2456953a2cbc21fde058d33fb2ad8d59e72a35,0x7dab328cf8c8cf7567a96e3e74ebd5aff2a0ae3d,"360,000,000.00","360,000,000.00",NaN,NaN
27,0x2f2456953a2cbc21fde058d33fb2ad8d59e72a35,"1,108,241,744.15",36.94,0x2f2456953a2cbc21fde058d33fb2ad8d59e72a350xc25d4d40dac5e0b8051fa3f82276908de3fc41d9ecf10887edf6f5eb3e22d74d,0xc25d4d40dac5e0b8051fa3f82276908de3fc41d9ecf10887edf6f5eb3e22d74d,2020-08-11 23:08:29,transfer,0x2f2456953a2cbc21fde058d33fb2ad8d59e72a35,0xb701dec4186c8c7f1a7047b33f9f9a39febb1db1,"6,000,000.00","6,000,000.00",NaN,NaN
28,0x2f2456953a2cbc21fde058d33fb2ad8d59e72a35,"1,108,241,744.15",36.94,0x2f2456953a2cbc21fde058d33fb2ad8d59e72a350x9563be30c250b56dbb0917300b80c220e422625269afaa91dbb281d4ed3a9dfd,0x9563be30c250b56dbb0917300b80c220e422625269afaa91dbb281d4ed3a9dfd,2020-08-05 10:27:29,transfer,0xeaa0993e1d21c2103e4f172a20d29371fbaf6d06,0x2f2456953a2cbc21fde058d33fb2ad8d59e72a35,"3,000,000,000.00","-3,000,000,000.00",NaN,NaN
1710,0x08f14cfcc8f0b67bfac70d3f93623775577e0909,"21,149,927.87",0.70,0x08f14cfcc8f0b67bfac70d3f93623775577e09090xb495559d52737757eddcc91fe0de8a51fab7aef09af27494b5f0c322941da8f6,0xb495559d52737757eddcc91fe0de8a51fab7aef09af27494b5f0c322941da8f6,2021-11-17 04:45:56,transfer,0x002d91c86d026319ba0d159c84ffea03bc00adf9,0x08f14cfcc8f0b67bfac70d3f93623775577e0909,"76,185.00","-76,185.00",NaN,NaN
4284,0xb701dec4186c8c7f1a7047b33f9f9a39febb1db1,"12,000,000.00",0.40,0xb701dec4186c8c7f1a7047b33f9f9a39febb1db10xc25d4d40dac5e0b8051fa3f82276908de3fc41d9ecf10887edf6f5eb3e22d74d,0xc25d4d40dac5e0b8051fa3f82276908de3fc41d9ecf10887edf6f5eb3e22d74d,2020-08-11 23:08:29,transfer,0x2f2456953a2cbc21fde058d33fb2ad8d59e72a35,0xb701dec4186c8c7f1a7047b33f9f9a39febb1db1,"6,000,000.00","-6,000,000.00",NaN,NaN
6768,0x5366dc49e76ba0173921c636aa23dcd00e4b296b,"1,500,000.00",0.05,0x5366dc49e76ba0173921c636aa23dcd00e4b296b0x186374b502fbe1e1f7842c6f603579ce221ae674f8be9e907c39d3c7a85ca4d4,0x186374b502fbe1e1f7842c6f603579ce221ae674f8be9e907c39d3c7a85ca4d4,2020-08-13 20:43:14,transfer,0x2f2456953a2cbc21fde058d33fb2ad8d59e72a35,0x5366dc49e76ba0173921c636aa23dcd00e4b296b,"50,000,000.00","-50,000,000.00",NaN,NaN
12648,0x35e39e0a8e16b4c7ab1d3f32ad5ca6bfafb90d06,"258,050.28",0.01,0x35e39e0a8e16b4c7ab1d3f32ad5ca6bfafb90d060x25590d0792a4008266676a5c71c64d083301f3e702e5ce653c5927f921282dd9,0x25590d0792a4008266676a5c71c64d083301f3e702e5ce653c5927f921282dd9,2021-11-19 16:27:43,transfer,0x35e39e0a8e16b4c7ab1d3f32ad5ca6bfafb90d06,0x08a2103a11e898ec31709fff6883d1658e20071d,453.97,453.97,NaN,NaN
12926,0x35e39e0a8e16b4c7ab1d3f32ad5ca6bfafb90d06,"258,050.28",0.01,0x35e39e0a8e16b4c7ab1d3f32ad5ca6bfafb90d060xdf591ace8f377d1d05b8cb71e30ea1d36b06ff759ea71d74ce6c101f441c00bf,0xdf591ace8f377d1d05b8cb71e30ea1d36b06ff759ea71d74ce6c101f441c00bf,2021-10-28 10:08:53,transfer,0x35e39e0a8e16b4c7ab1d3f32ad5ca6bfafb90d06,0x3507e4978e0eb83315d20df86ca0b976c0e40ccb,"57,145.00","57,145.00",NaN,NaN
17057,0x4da0eda15a164e78fddc0eddbee9295a78aa230e,"167,151.02",0.01,0x4da0eda15a164e78fddc0eddbee9295a78aa230e0xd8b0e5d7bf5194dbb7536f742e82a80d1510609b7dc81fdcce0777d900e3531e,0xd8b0e5d7bf5194dbb7536f742e82a80d1510609b7dc81fdcce0777d900e3531e,2021-11-19

## Holding Period Checks:

In [32]:
wallet_level_data['holding_period_month'].value_counts(bins = [0,1, 2, 3,6,12,16])

(-0.001, 1.0]    116
(6.0, 12.0]       27
(1.0, 2.0]        18
(2.0, 3.0]        17
(12.0, 16.0]      12
(3.0, 6.0]         9
Name: holding_period_month, dtype: int64

## Net Cash Flow by Wallet

In [108]:
df.groupby(['holders_address']).sum().sort_values('value_in_USD')


,abs_value,value,USD_price_at_timestamp,value_in_USD
holders_address,,,,
0xf977814e90da44bfa03b6295a0616a897441acec,"245,528,132.04","-44,884,670.50",18.74,"-116,508,068.50"
0x8d6f396d210d385033b348bcae9e4f9ea4e045bd,"28,360,000.00","-28,360,000.00",11.67,"-91,885,437.26"
0x08f14cfcc8f0b67bfac70d3f93623775577e0909,"302,820,614.32","-21,100,594.32","2,382.00","-45,845,967.88"
0x326d9f47ba49bbaac279172634827483af70a601,"60,000,005.00","-10,600,005.00",64.10,"-44,975,392.27"
0x37d493e32ca5ef914389c8cb2e33e81805309d96,"60,016,739.35","-9,003,697.59",43.80,"-37,552,865.47"
...,...,...,...,...
0xdd10c6e0b95ea74a6e3fcb1b7f119681e83f2dd8,"12,400,000.00","-2,500,000.00",47.57,"15,642,017.98"
0x3f148612315aae2514ac630d6faf0d94b8cd8e33,"13,304,376.41","3,708,819.55",388.00,"16,069,236.60"
0xb15bfc3c04a726dc6df1bd8d658e1a1e43a9af33,"8,617,165.50","2,683,369.36",749.19,"21,980,762.08"


In [109]:
df.groupby(['holders_address']).sum().sort_values('value')

,abs_value,value,USD_price_at_timestamp,value_in_USD
holders_address,,,,
0x2f2456953a2cbc21fde058d33fb2ad8d59e72a35,"4,838,710,063.85","-1,161,289,936.15",7.35,"160,665,045.12"
0x5f5b8942ae6325227d5d82e5759e837271fa2a67,"1,037,000,000.00","-523,000,000.00",2.13,"-2,347,350.65"
0x8ffa64fb50559c3ff09a1022b84b2c5233ed8068,"481,447,009.37","-281,512,136.63",18.50,"-1,519,098.94"
0x07d2601739709c25fe0afd50ec961ba589311caa,"225,000,000.00","-195,000,000.00",2.01,"-1,221,396.64"
0x5366dc49e76ba0173921c636aa23dcd00e4b296b,"516,306,912.49","-51,500,000.00",27.09,"-6,502,122.94"
...,...,...,...,...
0x2434e7e28b2b139511bc2b6f31e14ee1e6e03b4f,"9,714,833.56","986,118.36",215.98,"4,583,658.84"
0xc1ff7cea93d2324a76e1cb19c98d5e0a3a2b6b3f,"75,010,717.88","2,210,717.88","3,259.98","4,918,093.10"
0xb15bfc3c04a726dc6df1bd8d658e1a1e43a9af33,"8,617,165.50","2,683,369.36",749.19,"21,980,762.08"
